<a href="https://colab.research.google.com/github/jellyho/AI_Learning_jellyho/blob/main/Kaggle_1_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요 패키지 임포트

In [ ]:
import pandas as pd
import numpy as np

데이터 불러오기

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/jellyho/Kaggle_Projects/master/1.Titanic/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/jellyho/Kaggle_Projects/master/1.Titanic/test.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
data = train.copy()
data_test = test.copy()

data = data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
data_test = data_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

데이터 전처리 (Embarked와 Sex는 순서없는 정보이므로 숫자로 매핑, Age와 Fare은 값이 크므로 표준화를 통해 정보 줄이기.)

In [ ]:
#Age, Fare는 누락 된 값 평균내서 채우기
data['Age'] = data['Age'].fillna(data['Age'].mean(axis=0))
data['Fare'] = data['Fare'].fillna(data['Fare'].mean(axis=0))
data_test['Age'] = data_test['Age'].fillna(data_test['Age'].mean(axis=0))
data_test['Fare'] = data_test['Fare'].fillna(data_test['Fare'].mean(axis=0))

#Embarked 숫자로 처리
data['Embarked'] = data['Embarked'].map({'C':0, 'Q':1, 'S':2})
data_test['Embarked'] = data_test['Embarked'].map({'C':0, 'Q':1, 'S':2})

#성별 숫자로 처리
data['Sex'] = data['Sex'].map({'male':0, 'female':1})
data_test['Sex'] = data_test['Sex'].map({'male':0, 'female':1})

m_Age = data['Age'].mean(axis=0)
std_Age = data['Age'].std(axis=0)
m_Fare = data['Fare'].mean(axis=0)
std_Fare = data['Fare'].std(axis=0)

data['Age'] = ( data['Age'] - m_Age ) / std_Age
data['Fare'] = ( data['Fare'] - m_Fare) / std_Fare
data_test['Age'] = ( data_test['Age'] - m_Age ) / std_Age
data_test['Fare'] = ( data_test['Fare'] - m_Fare) / std_Fare

data = data.dropna()

#필요한 데이터만 반환
X_train = data[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']].values
y_train = data['Survived'].values

X_test = data_test[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']].values


로지스틱 회귀를 통한 학습

In [38]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', multi_class='auto', penalty='l1', C=1.0, random_state=1)
lr.fit(X_train, y_train)
result = lr.predict(X_test)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=1, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

서포트 벡터 머신을 이용한 학습

In [44]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train, y_train)
result = svm.predict(X_test)

커널 SVM을 이용한 학습

In [46]:
svm2 = SVC(kernel='rbf', random_state=1, gamma=0.2, C=1.0)
svm2.fit(X_train, y_train)
result = svm2.predict(X_test)

In [47]:
Result = pd.DataFrame(columns=['Survived'], data=result)
Result.index = range(892, 1310, 1)
Result.index.name = 'PassengerId'
Result.to_csv('Result.csv')

현재 정확도 : 0.77990